In [15]:
import os
import configparser

import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import fiona

import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches

import requests
import json

In [16]:
project_root = os.path.abspath(os.getcwd())
root = os.path.abspath(os.path.join(os.getcwd(), ".."))
config_file =  os.path.join(root, "config.cfg")

# geodata for shapefiles
config = configparser.ConfigParser()
config.read(config_file)
gdata_root = config["geodata"]["path"]

In [71]:
#load berlin borders
bezirke= gpd.GeoDataFrame.from_file(os.path.join(gdata_root,'Berlin', 'bezirke.gpkg'), crs ='EPSG:25833').to_crs('EPSG:4326')
berlin_borders=gpd.GeoDataFrame(geometry=[bezirke.unary_union], crs ='EPSG:4326')

In [18]:
#load isochrones
iso = []
fname = os.path.join(gdata_root,'Berlin', 'water_wells_isochrones3600.gpkg')
layers = fiona.listlayers(fname)
for layer in layers:
    tmp_df = gpd.GeoDataFrame.from_file(fname, crs ='EPSG:4326',layer=layer)
    iso.append(tmp_df)        

iso = pd.concat(iso, axis=0, ignore_index=True)

In [19]:
#load wells
wells= gpd.GeoDataFrame.from_file(os.path.join(gdata_root,'Berlin', 'water_wells.gpkg'), crs ='EPSG:4326')

In [20]:
#use only suitable wells, making a reasonably broad assumption
suitable_wells=wells[(wells['drinking_water:legal']=='yes') | 
                     (wells['drinking_water']=='yes') |
                     (wells['emergency']=='drinking_water') |
                     (wells['network']=='Berliner Straßenbrunnen')]
len(suitable_wells)

1978

In [21]:
#select only isochrones belonging to these wells:
iso = iso[iso.id.isin(suitable_wells.id)]

In [22]:
#now merge individual geometries
unions=[]
for time in iso.time.unique():
    df_tmp=pd.DataFrame(data={'time':[time],'geometry':[iso[iso.time==time].unary_union]})
    unions.append(df_tmp)
unions=pd.concat(unions, ignore_index=True).sort_values(by='time').reset_index(drop=True)
unions = gpd.GeoDataFrame(unions, crs ='EPSG:4326', geometry="geometry")

In [24]:
#get the isochrone slices through the magic of vector operations
unions['prev_geometry'] = unions['geometry'].shift(1)
unions['geometry_difference'] = unions.apply(lambda row: row['geometry'].difference(row['prev_geometry']) if pd.notnull(row['prev_geometry']) else row['geometry'], axis=1)
unions.drop('prev_geometry', axis=1, inplace=True)

In [ ]:
#huh, it was too naive to think that works that easy - there are still regions where different isochrones overlap
unions.geometry_difference.explore()

In [26]:
#now we look for a difference between a given timeslice and all isochrones with a shorter time...
unions['geometry_difference'] = unions.apply(lambda row: row['geometry'].difference(unions[unions.time<row.time].unary_union) if pd.notnull(unions[unions.time<row.time].unary_union) else row['geometry'], axis=1)

C:\temp\ipykernel_3628\3244895051.py:2: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  unions['geometry_difference'] = unions.apply(lambda row: row['geometry'].difference(unions[unions.time<row.time].unary_union) if pd.notnull(unions[unions.time<row.time].unary_union) else row['geometry'], axis=1)


In [ ]:
#huh, it was too naive to think that works that easy - there are still regions where different isochrones overlap
unions.geometry_difference.explore()

In [30]:
unions.head()

,time,geometry,geometry_difference
0,60,"MULTIPOLYGON (((13.12865 52.38867, 13.12968 52...","MULTIPOLYGON (((13.12865 52.38867, 13.12968 52..."
1,120,"MULTIPOLYGON (((13.12784 52.38769, 13.12828 52...","MULTIPOLYGON (((13.12828 52.38866, 13.13001 52..."
2,180,"MULTIPOLYGON (((13.12858 52.38621, 13.12810 52...","MULTIPOLYGON (((13.12810 52.38641, 13.12732 52..."
3,240,"MULTIPOLYGON (((13.12879 52.38544, 13.12853 52...","MULTIPOLYGON (((13.12853 52.38544, 13.12708 52..."
4,300,"MULTIPOLYGON (((13.12590 52.38579, 13.12603 52...","MULTIPOLYGON (((13.12603 52.38613, 13.12828 52..."


In [77]:
slices=gpd.GeoDataFrame(data={'time':unions.time,'geometry':unions.geometry_difference},crs='EPSG:4326')
#cut on the city border
slices=slices.overlay(berlin_borders,how='intersection')

In [79]:
#save data
slices.to_file(os.path.join(gdata_root,'Berlin', 'water_wells_isochrones_slices.gpkg'), driver='GPKG')
slices[["time", "geometry"]].to_crs(4326).to_file(os.path.join(gdata_root,'Berlin', 'water_wells_isochrones_slices.json'))
suitable_wells[["id", "geometry"]].to_crs(4326).to_file(os.path.join(gdata_root,'Berlin', 'suitable_wells.json'))